# Installation

In [ ]:
# clone nemo
!git clone https://github.com/AI4Bharat/NeMo.git -b nemo-v2

# conda install -c nvidia cuda-nvprof=12.2 # Cuda version
!pip install packaging
!pip install huggingface_hub==0.23.2

# install NeMo
%cd NeMo
!bash reinstall.sh

Cloning into 'NeMo'...
remote: Enumerating objects: 79391, done.
remote: Counting objects: 100% (311/311), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 79391 (delta 234), reused 232 (delta 201), pack-reused 79080 (from 1)
Receiving objects: 100% (79391/79391), 140.95 MiB | 17.25 MiB/s, done.
Resolving deltas: 100% (57651/57651), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 26.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.24.6
    Uninstalling huggingface-hub-0.24.6:
      Successfully uninstalled huggingface-hub-0.24.6
/content/NeMo
  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Uninstalling stuff
Installing nemo
Obtaining file:///content/NeMo
  Installing build dependencies ...

# Necessary imports

In [1]:
import torch
import soundfile as sf
import nemo.collections.asr as nemo_asr
import IPython

ModuleNotFoundError: No module named 'nemo'

# Loading the model
Once you download the .nemo checkpoint, you can load it here

In [ ]:
import torch
import zipfile
import os
# Path to the .nemo file
checkpoint = "/content/ai4b_indicConformer_hi.nemo"




In [ ]:
!pip install fastapi

In [ ]:
from fastapi import FastAPI, File, UploadFile

In [ ]:
!pip install uvicorn

In [ ]:
import uvicorn
import os
app = FastAPI()

In [ ]:
Language_Id = "hi"

In [ ]:
@app.post("/transcribe/")
async def transcribe(file: UploadFile = File(...)):
    # Save the uploaded file
    file_location = "/content/audio.mpr" # Upload your audio file and replace it with your filepath
    with open(file_location, "wb+") as file_object:
        file_object.write(file.file.read())

    # Convert MP4 to WAV if necessary
    if file.filename.endswith('.mp4'):
        wav_file_location = convert_mp4_to_wav(file_location)
        transcription = transcribe_audio(wav_file_location)
        os.remove(wav_file_location)  # Remove the temporary WAV file
    else:
        transcription = transcribe_audio(file_location)

    # Remove the temporary file
    os.remove(file_location)

    # Return the transcription
    return {"transcription": transcription}

In [ ]:
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

#Since task -5 is a combination of stacks used in task 2, 3 and 4, We will implement it here itself

# Task - 5

In [ ]:
import streamlit as st
import requests
import soundfile as sf
import torch
import nemo.collections.asr as nemo_asr
from pydantic import BaseModel
from typing import Dict, List
import json
from sentence_transformers import SentenceTransformer, util
import torch.nn as nn
import faiss
import numpy as np
import openai

In [ ]:
# Sarvam Translation API
TRANSLATION_API_KEY = "443f8d9a-23b8-43a4-99e8-a966fe61ff65"
TRANSLATION_API_ENDPOINT = "https://api.sarvam.ai/translate"


In [ ]:
# Faiss Vector DB
VECTOR_DB_INDEX = faiss.IndexFlatL2(128)

In [ ]:
# LLM API
LLM_API_KEY = # use your suitable api key(here, I have used openai api)


In [ ]:
# Create a Streamlit app
st.title("Voice-Enabled Chatbot")

# Upload audio file
uploaded_file = st.file_uploader("Choose an audio file", type=["mp3", "wav", "mp4"])

In [ ]:
# Transcribe audio button
if st.button("Transcribe Audio"):
    if uploaded_file is not None:
        # Save the uploaded audio file
        with open("audio.mp4", "wb") as file:
            file.write(uploaded_file.getbuffer())

        # Transcribe the audio file using ASR endpoint
        transcription_response = requests.post(ASR_ENDPOINT, files={'file': open("audio.mp4", 'rb')})
        transcription = transcription_response.json()["transcription"]

        # Translate the text to English using Sarvam Translation API
        translation_payload = {
            "input": transcription,
            "source_language_code": "hi-IN",
            "target_language_code": "en-US",
            "speaker_gender": "Male",
            "mode": "formal",
            "model": "mayura:v1",
            "enable_preprocessing": True
        }
        translation_headers = {
            'API-Subscription-Key': TRANSLATION_API_KEY,
            'Content-Type': 'application/json'
        }
        translation_response = requests.post(TRANSLATION_API_ENDPOINT, headers=translation_headers, json=translation_payload)
        translated_text = translation_response.json()["output"]

        # Compute the vector representation of the translated text
        vector = sentence_model.encode(translated_text)

        # Search the Faiss Vector DB for the top-2 closest chunks
        distances, indices = VECTOR_DB_INDEX.search(np.array([vector]), k=2)

        # Get the answer for the question using OpenAI LLM API
        openai.api_key = OPENAI_API_KEY
        response = openai.Completion.create(
            engine="davinci",
            prompt=translated_text,
            max_tokens=2048,
            temperature=0.5,
            frequency_penalty=0,
            presence_penalty=0
        )
        answer = response.choices[0].text

        # Display the answer
        st.write("Answer:")
        st.write(answer)
    else:
        st.write("Please upload an audio file")

In [ ]:
# Add vectors to the Faiss Vector DB
def add_vectors_to_db(vectors):
    VECTOR_DB_INDEX.add(vectors)

# Run the Streamlit app
if __name__ == "__main__":
    st.run()

# Without streamlit

In [ ]:
# Define the request body for the chatbot endpoint
class ChatbotRequest(BaseModel):
    audio_file: UploadFile

# Load the sentence transformer model
sentence_model = SentenceTransformer('distilbert-base-nli-mean-tokens')

# Define the endpoint for the chatbot
@app.post("/chatbot/")
async def chatbot(request: ChatbotRequest):
    # Save the uploaded audio file
    file_location = f"temp/{request.audio_file.filename}"
    with open(file_location, "wb+") as file_object:
        file_object.write(request.audio_file.file.read())

    # Transcribe the audio file using ASR endpoint
    transcription_response = requests.post(ASR_ENDPOINT, files={'file': open(file_location, 'rb')})
    transcription = transcription_response.json()["transcription"]

    # Translate the text to English using Sarvam Translation API
    translation_payload = {
        "input": transcription,
        "source_language_code": "hi-IN",
        "target_language_code": "en-US",
        "speaker_gender": "Male",
        "mode": "formal",
        "model": "mayura:v1",
        "enable_preprocessing": True
    }
    translation_headers = {
        'API-Subscription-Key': TRANSLATION_API_KEY,
        'Content-Type': 'application/json'
    }
    translation_response = requests.post(TRANSLATION_API_ENDPOINT, headers=translation_headers, json=translation_payload)
    translated_text = translation_response.json()["output"]

    # Compute the vector representation of the translated text
    vector = sentence_model.encode(translated_text)

    # Search the Faiss Vector DB for the top-2 closest chunks
    distances, indices = VECTOR_DB_INDEX.search(np.array([vector]), k=2)

    # Get the answer for the question using OpenAI LLM API
    openai.api_key = OPENAI_API_KEY
    response = openai.Completion.create(
        engine="davinci",
        prompt=translated_text,
        max_tokens=2048,
        temperature=0.5,
        frequency_penalty=0,
        presence_penalty=0
    )
    answer = response.choices[0].text

    # Return the final answer
    return {"answer": answer}


In [ ]:
# Add vectors to the Faiss Vector DB
def add_vectors_to_db(vectors):
    VECTOR_DB_INDEX.add(vectors)

# Run the FastAPI app
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)